### Preparation Notebook - Future Flood


This notebook processes ISIMIP future forced hyrdological model outputs to calculate basin level future flood-frequency changes

In [22]:
# Import live code changes in
%load_ext autoreload
%autoreload 

import os

from pathlib import Path
from joblib import Parallel, delayed
import xarray as xr
import scipy.stats as stats
import numpy as np
import pandas as pd
import rasterio
from rasterio.transform import from_origin
from rasterstats import zonal_stats
import geopandas as gpd
from tqdm import tqdm
from itertools import product
import warnings 
warnings.filterwarnings("ignore", category=RuntimeWarning)

from sovereign.utils import pot_with_optimal_threshold, df_to_raster

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


##### 1. User Config

In [27]:
return_periods = [10, 20, 50, 75, 100, 200, 500] # return periods of interest 
# Sampling periods for EVA
sample_periods = {
    "historical": ("1965-01-01", "2014-12-31"),
    "2030": ("2005-01-01", "2054-12-31"),
    "2040": ("2015-01-01", "2064-12-31"),
    "2050": ("2025-01-01", "2074-12-31"),
    "2060": ("2035-01-01", "2084-12-31"),
    "2070": ("2045-01-01", "2094-12-31")
}
# For rasterization
EPOCHS = ['2030', '2040', '2050', '2060', '2070'] # what future epochs are we interested in?
SCENARIOS = ['ssp126', 'ssp370', 'ssp585'] # what climate scenarios are we intersted in?

##### 2. Set filepaths

In [3]:
root = Path.cwd().parent.parent # find project root
isimip_path = Path(os.path.join(root, 'inputs', 'flood', 'future', 'isimip', 'THA')) # this directory contains relevant ISIMIP layers
prelim_output_path = Path(os.path.join(root, 'outputs', 'flood', 'future', 'prelim')) 
basin_path = Path(os.path.join(root, 'inputs', 'boundaries', 'basins', 'BA_THA_lev06.shp'))
RASTER_DIR = Path(os.path.join(root, 'outputs', 'flood', 'future', 'maps')) # hwere to save the rasters
FUTURE_BASINS = Path(os.path.join(root, 'outputs', 'flood', 'future', 'basin_rp_shifts.csv')) # where to save the basin RP shifts

##### 3. Run POT EVA on ISIMIP layers (Parallelized)

In [5]:
def process_cell(lat, lon, dis_data, sample_periods, return_periods, model, climate_scenario):
    """Process a single grid cell across all periods and return periods."""
    cell_results = []
    for period_name, (start_date, end_date) in sample_periods.items():
        period_data = dis_data.sel(time=slice(start_date, end_date)).values
        if np.std(period_data) == 0.0:
            continue
        for return_period in return_periods:
            try:
                best, diagnostics = pot_with_optimal_threshold(
                    x=period_data,
                    candidate_ps=[0.90, 0.92, 0.94, 0.96, 0.97, 0.98, 0.99],
                    return_period=return_period
                )
                if best is None:
                    continue
                cell_results.append({
                    "latitude": lat,
                    "longitude": lon,
                    "return_level": best['qT'],
                    "log_return_level": best['log_qT'],
                    "std_log_return_level": best['var_log_qT'] ** 0.5,
                    "q5": np.exp(best['log_qT'] - 1.96 * best['var_log_qT'] ** 0.5),
                    "q95": np.exp(best['log_qT'] + 1.96 * best['var_log_qT'] ** 0.5),
                    "xi_hat": best['xi'],
                    "sigma_hat": best['sigma'],
                    "selected_threshold": best['p'],
                    "threshold_discharge": best['u'],
                    "num_exceedances": best['n_exc'],
                    "ks_pvalue": best['ks_pvalue'],
                    "return_period": return_period,
                    "model": model,
                    "period": period_name,
                    "period_start_date": start_date,
                    "period_end_date": end_date,
                    "climate_scenario": climate_scenario,
                })
            except Exception as e:
                print(f"Error at lat {lat}, lon {lon}: {e}")
    return cell_results

# Initialize results dataframe
pot_results = pd.DataFrame()
models = [f for f in os.listdir(isimip_path) if f.endswith('.nc')]

for model in models:
    parquet_path = os.path.join(prelim_output_path, f'{model}.parquet')
    if os.path.exists(parquet_path):
        print(f"Skipping model {model}, results already exist.")
        pot_results = pd.concat([pot_results, pd.read_parquet(parquet_path)], ignore_index=True)
        continue

    print(f"Processing model: {model}")
    dataset = xr.open_dataset(os.path.join(isimip_path, model))
    latitudes = dataset['lat'].values
    longitudes = dataset['lon'].values
    climate_scenario = model.split('_')[-1].replace('.nc', '')

    dis = dataset['dis']

    # Run cell-level processing in parallel
    results_nested = Parallel(n_jobs=-1, backend='loky')(
        delayed(process_cell)(
            lat, lon,
            dis.sel(lat=lat, lon=lon, method="nearest"),
            sample_periods, return_periods, model, climate_scenario
        )
        for lat, lon in[(lat, lon) for lat in latitudes for lon in longitudes]
    )

    # Flatten list of lists
    flat_results = [row for cell in results_nested for row in cell]
    model_results = pd.DataFrame(flat_results)

    Path(prelim_output_path).mkdir(parents=True, exist_ok=True)
    model_results.to_parquet(parquet_path)
    pot_results = pd.concat([pot_results, model_results], ignore_index=True)

pot_results = pot_results.dropna()

Skipping model THA_dis_jules-w2_gfdl-esm4_ssp126.nc, results already exist.
Skipping model THA_dis_jules-w2_gfdl-esm4_ssp370.nc, results already exist.
Skipping model THA_dis_jules-w2_gfdl-esm4_ssp585.nc, results already exist.
Skipping model THA_dis_jules-w2_ipsl-cm6a-lr_ssp126.nc, results already exist.
Skipping model THA_dis_jules-w2_ipsl-cm6a-lr_ssp370.nc, results already exist.
Skipping model THA_dis_jules-w2_ipsl-cm6a-lr_ssp585.nc, results already exist.
Skipping model THA_dis_jules-w2_mpi-esm1-2-hr_ssp126.nc, results already exist.
Skipping model THA_dis_jules-w2_mpi-esm1-2-hr_ssp370.nc, results already exist.
Skipping model THA_dis_jules-w2_mpi-esm1-2-hr_ssp585.nc, results already exist.
Skipping model THA_dis_jules-w2_mri-esm2-0_ssp126.nc, results already exist.
Skipping model THA_dis_jules-w2_mri-esm2-0_ssp370.nc, results already exist.
Skipping model THA_dis_jules-w2_mri-esm2-0_ssp585.nc, results already exist.
Skipping model THA_dis_jules-w2_ukesm1-0-ll_ssp126.nc, results a

##### 5. Combine results to clean dataframe

In [11]:
baseline_levels = pot_results[pot_results['period'] == 'historical'][[
    'latitude', 'longitude', 'climate_scenario', 'model', 'return_period', 'return_level', 'q5', 'q95', 'xi_hat', 'sigma_hat', 'selected_threshold']
    ]
future_with_baseline =pd.merge(
    pot_results,
    baseline_levels,
    on=['latitude', 'longitude', 'climate_scenario', 'model', 'return_period'],
    suffixes=('', '_baseline')
)
# Calculate multipliers
future_with_baseline['multiplier'] = future_with_baseline['return_level'] / future_with_baseline['return_level_baseline']
future_with_baseline['multiplier_q5'] = future_with_baseline['q5'] / future_with_baseline['return_level_baseline']
future_with_baseline['multiplier_q95'] = future_with_baseline['q95'] / future_with_baseline['return_level_baseline']

##### 6. Calculate adjusted return periods

In [18]:
def get_adjusted_return_period(
    xi_hat,
    sigma_hat,
    threshold_discharge,  
    return_level_scenario,
    return_level_baseline,
    baseline_return_period
):
    # Basic checks
    vals = [xi_hat, sigma_hat, threshold_discharge, return_level_scenario, return_level_baseline, baseline_return_period]
    if any(v is None or np.isnan(v) for v in vals):
        return np.nan
    if sigma_hat <= 0 or baseline_return_period <= 0:
        return np.nan

    if abs(xi_hat) < 1e-6:
        return float(baseline_return_period * np.exp((return_level_baseline - return_level_scenario) / sigma_hat))

    # ξ ≠ 0: general GP case
    num = 1 + xi_hat * (return_level_baseline - threshold_discharge) / sigma_hat
    den = 1 + xi_hat * (return_level_scenario - threshold_discharge) / sigma_hat

    # Domain check for the GP (must be > 0)
    if num <= 0 or den <= 0:
        return np.nan

    ratio = (num / den) ** (1.0 / xi_hat)
    return float(baseline_return_period * ratio)

future_with_baseline['adjusted_return_period'] = future_with_baseline.apply(
    lambda row: get_adjusted_return_period(
        xi_hat=row['xi_hat'],
        sigma_hat=row['sigma_hat'],
        threshold_discharge=row['selected_threshold'],
        return_level_scenario=row['return_level'],
        return_level_baseline=row['return_level_baseline'],
        baseline_return_period=row['return_period']
    ),
    axis=1
)

future_with_baseline['adjusted_return_period_q5'] = future_with_baseline.apply(
    lambda row: get_adjusted_return_period(
        xi_hat=row['xi_hat'],
        sigma_hat=row['sigma_hat'],
        threshold_discharge=row['selected_threshold'],
        return_level_scenario=row['q5'],
        return_level_baseline=row['return_level_baseline'],
        baseline_return_period=row['return_period']
    ),
    axis=1
)

future_with_baseline['adjusted_return_period_q95'] = future_with_baseline.apply(
    lambda row: get_adjusted_return_period(
        xi_hat=row['xi_hat'],
        sigma_hat=row['sigma_hat'],
        threshold_discharge=row['selected_threshold'],
        return_level_scenario=row['q95'],
        return_level_baseline=row['return_level_baseline'],
        baseline_return_period=row['return_period']
    ),
    axis=1
)
# Clean the dataframe for next step (extract climate model and hydro model names and add to column)
future_with_baseline['noext'] = future_with_baseline['model'].str.replace('.nc', '', regex=False)
parts = future_with_baseline['noext'].str.split('_', expand=True) # split by '_'
future_with_baseline['hydro'] = parts[2] # add a specific hydro model column
future_with_baseline['climate'] = parts[3] # add a specific climate model column
future_with_baseline = future_with_baseline.drop(columns=['noext']) # clean up

##### 7. Create ISIMIP future RP change rasters

In [25]:
# Find all unique hydro model and climate model pairs in the dataframe
model_pairs = (
    future_with_baseline[['hydro', 'climate']]
    .drop_duplicates()
    .itertuples(index=False, name=None)
)
model_pairs = list(model_pairs) # Make it a list

# Create raster directory if it doesn't already exoist
RASTER_DIR.mkdir(parents=True, exist_ok=True)

# Build full combination list
all_jobs = list(product(model_pairs, SCENARIOS, EPOCHS, return_periods))

for (hydro, clim), scen, epoch, rp in tqdm(all_jobs, desc="Rasterizing", unit="file"):
    # Set output filepath
    out_name = f"{hydro}_{clim}_{scen}_{epoch}_rp{rp:03d}.tif"
    out_path = Path(os.path.join(RASTER_DIR, out_name))
    # Skip if file already exists
    if out_path.exists():
        continue
        
    # Extract dataframe subset
    sub = future_with_baseline[
        (future_with_baseline['hydro'] == hydro) &
        (future_with_baseline['climate'] == clim) & 
        (future_with_baseline['climate_scenario'] == scen) & 
        (future_with_baseline['period'] == epoch) & 
        (future_with_baseline['return_period'] == rp)
    ]
    if sub.empty:
        continue
        
    # Convert to raster
    df_to_raster(sub, out_path)

Rasterizing: 100%|█████████████████████████████████████████████████████████████████| 525/525 [00:47<00:00, 11.09file/s]


##### 8. Calculate hydrological basin-level RP change statistics (running zonal stats on ISIMIP rasters)

In [29]:
# Load the basins file
basins = gpd.read_file(basin_path)
basins = basins[['HYBAS_ID', 'geometry']] # just extract basin ID and geometry info
basins = basins.rename(columns={'HYBAS_ID': 'HB_L6'}) # rename basin ID to align with remaining workflow (working at hydrobasin level 6)

records = [] # empty list to save results to
# Loop over all rasters
for tif_path in tqdm(sorted(RASTER_DIR.glob("*.tif")), desc="Running Zonal Stats"):
    stem = tif_path.stem  # e.g. "cwatm_gfdl-esm4_ssp126_2030_rp010"
    parts = stem.split("_")
    if len(parts) != 5:
        print(f"Skipping unexpected filename: {stem}")
        continue

    hydro, clim, scen, epoch, rp_str = parts
    rp = int(rp_str.replace("rp", ""))

    # Zonal stats: mean per basin
    zs = zonal_stats(
        basins,
        tif_path,
        stats=["mean"],
        nodata=-9999,
        all_touched=True   # calculate stats on any cells that touch region
    )
    means = [z["mean"] for z in zs]

    tmp = pd.DataFrame({
        'HB_L6': basins['HB_L6'].values,
        'hydro': hydro,
        'climate': clim,
        'climate_scenario': scen,
        'period': int(epoch),
        'return_period': rp,
        "basin_mean_value": means,
    })

    records.append(tmp)

# Combine everything
future_rp_shifts = pd.concat(records, ignore_index=True)

Running Zonal Stats: 100%|███████████████████████████████████████████████████████████| 525/525 [18:17<00:00,  2.09s/it]


##### 9. Create hydrological basin return period change dataframe and save 

In [31]:
# Calculate stats across climate models for each hydro model and scenario:epoch:rp combination
grouped = (future_rp_shifts.groupby(['HB_L6', 'hydro', 'climate_scenario', 'period', 'return_period'])
    .agg(q95=("basin_mean_value", lambda x: x.quantile(0.05)),
        q50=("basin_mean_value", lambda x: x.quantile(0.50)),
        q05=("basin_mean_value", lambda x: x.quantile(0.95)),
        mean=("basin_mean_value", np.mean)))
grouped.columns.name = "stat"
# Convert to stacked dataframe for saving
long_stats = (
    grouped
    .stack()
    .reset_index(name="new_rp_value")
)
# Save to CSV
long_stats.to_csv(FUTURE_BASINS)

C:\Users\Mark.DESKTOP-UFHIN6T\AppData\Local\Temp\ipykernel_19632\3398958568.py:2: FutureWarning: The provided callable <function mean at 0x00000177B48CE820> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  grouped = (future_rp_shifts.groupby(['HB_L6', 'hydro', 'climate_scenario', 'period', 'return_period'])
